# Example: Conditional Parameter Grids

This example shows the usage of `PyExperimenter` with a conditional parameter grid. We will programmatically define the parameter combinations of a support vector machine, instead of generating the entire cartesian product from the parameters defined in the config file.  

To execute this notebook you need to install:
```
pip install py_experimenter
pip install scikit-learn
```

## Experiment Configuration File
This notebook shows an example execution of `PyExperimenter` based on an experiment configuration file. Further explanation about the usage of `PyExperimenter` can be found in the [documentation](https://tornede.github.io/py_experimenter/usage.html). Here, we only define keyfields and resultfields and do not set the parameter values in the experiment configuration file as we will create the parameter grid programmatically.

In [4]:
import os

content = """
PY_EXPERIMENTER:
  n_jobs: 1

  Database:
    provider: sqlite
    database: py_experimenter
    table: 
      name: example_conditional_grid
      keyfields:
        dataset:
          type: VARCHAR(50)
        cross_validation_splits:
          type: int
        seed: 
          type: int
        kernel: 
          type: VARCHAR(50)
        gamma: 
          type: VARCHAR(50)
        degree: 
          type: VARCHAR(50)
        coef0: 
          type: VARCHAR(50)
      result_timestamps: false
      resultfields:
        train_f1: DECIMAL
        train_accuracy: DECIMAL
        test_f1: DECIMAL
        test_accuracy: DECIMAL

  CUSTOM:
    path: sample_data
"""

# Create config directory if it does not exist
if not os.path.exists('config'):
    os.mkdir('config')
    
# Create config file
experiment_configuration_file_path = os.path.join('config', 'example_conditional_grid.yml')
with open(experiment_configuration_file_path, "w") as f: 
  f.write(content)

## Defining the execution function

Next, the execution of a single experiment has to be defined. Note that this is a dummy example, which contains limited reasonable code. It is meant to show the core functionality of the PyExperimenter. 

The method is called with the parameters, i.e. `keyfields`, of a database entry. The results are meant to be processed to be written into the database, i.e. as `resultfields`. 

In [5]:
import random

import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import cross_validate
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

from py_experimenter.experimenter import PyExperimenter
from py_experimenter.result_processor import ResultProcessor

from time import sleep
from random import randint

def run_svm(parameters: dict, result_processor: ResultProcessor, custom_config: dict):
    sleep(randint(0,5))
    seed = parameters['seed']
    random.seed(seed)
    np.random.seed(seed)

    data = load_iris()

    X = data.data
    y = data.target

    # Create Support Vector Machine with parameters dependent on the kernel
    kernel = parameters['kernel']
    if kernel == 'linear':
        svc = SVC(kernel=parameters['kernel'])
    elif kernel == 'poly':
        svc = SVC(kernel=parameters['kernel'], gamma=parameters['gamma'], coef0=parameters['coef0'], degree=parameters['degree'])
    elif kernel == 'rbf':
        svc = SVC(kernel=parameters['kernel'], gamma=parameters['gamma'])

    svc = SVC()

    model = make_pipeline(StandardScaler(), svc)  

    if parameters['dataset'] != 'iris':
        raise ValueError("Example error")

    scores = cross_validate(model, X, y, 
        cv=parameters['cross_validation_splits'],
        scoring=('accuracy', 'f1_micro'),
        return_train_score=True
    )
    
    result_processor.process_results({
        'train_f1': np.mean(scores['train_f1_micro']),
        'train_accuracy': np.mean(scores['train_accuracy'])
    })

    result_processor.process_results({
        'test_f1': np.mean(scores['test_f1_micro']),
        'test_accuracy': np.mean(scores['test_accuracy'])})

## Executing PyExperimenter

The actual execution of the PyExperimenter is done in multiple steps. 

### Initialize PyExperimenter
The PyExperimenter is initialized with the previously created configuration file. Additionally, `PyExperimenter` is given a `name`, i.e. job id, which is especially useful for parallel executions of multiple experiments on HPC. 

In [6]:
from py_experimenter.experimenter import PyExperimenter

experimenter = PyExperimenter(experiment_configuration_file_path=experiment_configuration_file_path, name="SVM_experimenter_01")

2024-02-27 09:11:53,745  | py-experimenter - WARNING  | No values given for keyfield dataset
2024-02-27 09:11:53,747  | py-experimenter - WARNING  | No values given for keyfield cross_validation_splits
2024-02-27 09:11:53,748  | py-experimenter - WARNING  | No values given for keyfield seed
2024-02-27 09:11:53,750  | py-experimenter - WARNING  | No values given for keyfield kernel
2024-02-27 09:11:53,752  | py-experimenter - WARNING  | No values given for keyfield gamma
2024-02-27 09:11:53,754  | py-experimenter - WARNING  | No values given for keyfield degree
2024-02-27 09:11:53,755  | py-experimenter - WARNING  | No values given for keyfield coef0
2024-02-27 09:11:53,757  | py-experimenter - INFO     | Found 7 keyfields
INFO:py-experimenter:Found 7 keyfields
2024-02-27 09:11:53,758  | py-experimenter - WARNING  | No logtables given
2024-02-27 09:11:53,760  | py-experimenter - WARNING  | No custom section defined in config
2024-02-27 09:11:53,761  | py-experimenter - WARNING  | No cod

### Fill Table

The table is filled programmatically using the  `fill_table_from_combination()` method. We first generate the fixed parameter combinations for each kernel of the SVM in the first three lines.
* For the `rbf` kernel, we need to set values for the `gamma` parameter. The degree and `coef0` parameter are not present in this kernel, so we set these to `'nan'`.
* For the `poly` kernel, we need to set the `gamma`, the degree as well as the `coef0` parameter.
* For the `linear` kernel, we do not need to set any parameters, so all of them are set to `'nan'`.

Afterwards, we combine these with the seed, the dataset and the cross_validation_splits parameters, which are present for all experiment runs. Thus, these are not set unconditionally.

 

Note that the table can easily be obtained as `pandas.Dataframe` via `experimenter.get_table()`.

In [7]:
# Create parameter configurations for each kernel
combinations = [{'kernel': 'rbf', 'gamma': gamma, 'degree':None, 'coef0':None} for gamma in ['0.1','0.3']]
combinations += [{'kernel': 'poly', 'gamma': gamma, 'degree': degree, 'coef0': coef0} for gamma in ['0.1','0.3'] for degree in ['3','4'] for coef0 in ['0.0', '0.1']]
combinations += [{'kernel': 'linear','gamma': None, 'degree':None, 'coef0':None}]

# Fill experimenter
experimenter.fill_table_from_combination(parameters={'seed': ['1', '2', '3', '4', '5'], 
'dataset': ['iris'],
'cross_validation_splits': ['5'] },
fixed_parameter_combinations=combinations)

# showing database table
experimenter.get_table()

2024-02-27 09:11:53,861  | py-experimenter - INFO     | 55 rows successfully added to database. 0 rows were skipped.
INFO:py-experimenter:55 rows successfully added to database. 0 rows were skipped.


,ID,dataset,cross_validation_splits,seed,kernel,gamma,degree,coef0,creation_date,status,start_date,name,machine,train_f1,train_accuracy,test_f1,test_accuracy,end_date,error
0,1,iris,5,1,rbf,0.1,None,None,2024-02-27 08:57:52,done,2024-02-27 08:57:52,SVM_experimenter_01,Worklaptop,1.0,1.0,1.0,1.0,2024-02-27 08:57:59,None
1,2,iris,5,1,rbf,0.3,None,None,2024-02-27 08:57:52,done,2024-02-27 08:58:00,SVM_experimenter_01,Worklaptop,1.0,1.0,1.0,1.0,2024-02-27 08:58:05,None
2,3,iris,5,1,poly,0.1,3,0.0,2024-02-27 08:57:52,done,2024-02-27 08:58:05,SVM_experimenter_01,Worklaptop,1.0,1.0,1.0,1.0,2024-02-27 08:58:11,None
3,4,iris,5,1,poly,0.1,3,0.1,2024-02-27 08:57:52,done,2024-02-27 08:58:11,SVM_experimenter_01,Worklaptop,1.0,1.0,1.0,1.0,2024-02-27 08:58:16,None
4,5,iris,5,1,poly,0.1,4,0.0,2024-02-27 08:57:52,done,2024-02-27 08:58:16,SVM_experimenter_01,Worklaptop,1.0,1.0,1.0,1.0,2024-02-27 08:58:22,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105,106,iris,5,5,poly,0.3,3,0.0,2024-02-27 09:11:53,created,NaT,None,None,NaN,NaN,NaN,NaN,NaT,None
106,107,iris,5,5,poly,0.3,3,0.1,2024-02-27 09:11:53,created,NaT,None,None,NaN,NaN,NaN,NaN,NaT,None
107,108,iris,5,5,poly,0.3,4,0.0,2024-02-27 09:11:53,created,NaT,None,None,NaN,NaN,NaN,NaN,NaT,None
108,109,iris,5,5,poly,0.3,4,0.1,2024-02-27 09:11:53,created,NaT,None,None,NaN,NaN,NaN,NaN,NaT,None


### Execute PyExperimenter
All experiments are executed one after the other by the same `PyExperimenter` due to `max_experiments=-1`. If just a single one or a predifined number of experiments should be executed, the `-1` has to be replaced by the according amount.
The first parameter, i.e. `run_svm`, relates to the actual method that should be executed with the given keyfields of the table. 

In [8]:
experimenter.execute(run_svm, max_experiments=-1)

# showing database table
experimenter.get_table() 

[codecarbon INFO @ 09:11:53] [setup] RAM Tracking...
[codecarbon INFO @ 09:11:53] [setup] GPU Tracking...
[codecarbon INFO @ 09:11:53] No GPU found.
[codecarbon INFO @ 09:11:53] [setup] CPU Tracking...
[codecarbon WARNING @ 09:11:53] No CPU tracking mode found. Falling back on CPU constant mode.


[codecarbon WARNING @ 09:11:55] We saw that you have a 12th Gen Intel(R) Core(TM) i7-1260P but we don't know it. Please contact us.
[codecarbon INFO @ 09:11:55] CPU Model on constant consumption mode: 12th Gen Intel(R) Core(TM) i7-1260P
[codecarbon INFO @ 09:11:55] >>> Tracker's metadata:
[codecarbon INFO @ 09:11:55]   Platform system: Linux-5.15.133.1-microsoft-standard-WSL2-x86_64-with-glibc2.35
[codecarbon INFO @ 09:11:55]   Python version: 3.9.0
[codecarbon INFO @ 09:11:55]   CodeCarbon version: 2.3.4
[codecarbon INFO @ 09:11:55]   Available RAM : 15.475 GB
[codecarbon INFO @ 09:11:55]   CPU count: 16
[codecarbon INFO @ 09:11:55]   CPU model: 12th Gen Intel(R) Core(TM) i7-1260P
[codecarbon INFO @ 09:11:55]   GPU count: None
[codecarbon INFO @ 09:11:55]   GPU model: None
[codecarbon INFO @ 09:12:02] Energy consumed for RAM : 0.000007 kWh. RAM Power : 5.803094387054443 W
[codecarbon INFO @ 09:12:02] Energy consumed for all CPUs : 0.000048 kWh. Total CPU Power : 42.5 W
[codecarbon INF

,ID,dataset,cross_validation_splits,seed,kernel,gamma,degree,coef0,creation_date,status,start_date,name,machine,train_f1,train_accuracy,test_f1,test_accuracy,end_date,error
0,1,iris,5,1,rbf,0.1,None,None,2024-02-27 08:57:52,done,2024-02-27 08:57:52,SVM_experimenter_01,Worklaptop,1.0,1.0,1.0,1.0,2024-02-27 08:57:59,None
1,2,iris,5,1,rbf,0.3,None,None,2024-02-27 08:57:52,done,2024-02-27 08:58:00,SVM_experimenter_01,Worklaptop,1.0,1.0,1.0,1.0,2024-02-27 08:58:05,None
2,3,iris,5,1,poly,0.1,3,0.0,2024-02-27 08:57:52,done,2024-02-27 08:58:05,SVM_experimenter_01,Worklaptop,1.0,1.0,1.0,1.0,2024-02-27 08:58:11,None
3,4,iris,5,1,poly,0.1,3,0.1,2024-02-27 08:57:52,done,2024-02-27 08:58:11,SVM_experimenter_01,Worklaptop,1.0,1.0,1.0,1.0,2024-02-27 08:58:16,None
4,5,iris,5,1,poly,0.1,4,0.0,2024-02-27 08:57:52,done,2024-02-27 08:58:16,SVM_experimenter_01,Worklaptop,1.0,1.0,1.0,1.0,2024-02-27 08:58:22,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105,106,iris,5,5,poly,0.3,3,0.0,2024-02-27 09:11:53,done,2024-02-27 09:16:42,SVM_experimenter_01,Worklaptop,1.0,1.0,1.0,1.0,2024-02-27 09:16:51,None
106,107,iris,5,5,poly,0.3,3,0.1,2024-02-27 09:11:53,done,2024-02-27 09:16:51,SVM_experimenter_01,Worklaptop,1.0,1.0,1.0,1.0,2024-02-27 09:17:00,None
107,108,iris,5,5,poly,0.3,4,0.0,2024-02-27 09:11:53,done,2024-02-27 09:17:00,SVM_experimenter_01,Worklaptop,1.0,1.0,1.0,1.0,2024-02-27 09:17:08,None
108,109,iris,5,5,poly,0.3,4,0.1,2024-02-27 09:11:53,done,2024-02-27 09:17:08,SVM_experimenter_01,Worklaptop,1.0,1.0,1.0,1.0,2024-02-27 09:17:17,None


### CodeCarbon
Note that `CodeCarbon` is activated by default, collecting information about the carbon emissions of each experiment. Have a look at our [general usage example](https://tornede.github.io/py_experimenter/examples/example_general_usage.html) and the according [documentation of CodeCarbon fields](https://tornede.github.io/py_experimenter/usage.html#codecarbon-fields) for more information.